In [126]:
import pandas as pd
import numpy as np

df = pd.read_csv("airlines.csv", nrows=5000)
# df_copy = df_copy.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
# df_copy.loc[df_copy.sample(frac=0.1).index, "Sex"] = np.nan
df.loc[df.sample(frac=0.1).index, "airportfrom"] = np.nan
df = df.drop(['flight'], axis=1)
df.isnull().sum()
nanInd = df.loc[pd.isna(df['airportfrom']), :].index
# df
# df_orig.info()

In [127]:
# cat_variables = df_copy[['Sex', 'Embarked']]
cat_variables = df[['airline', 'airportfrom', 'airportto']]
cat_dummies = pd.get_dummies(cat_variables, drop_first=True)
cat_dummies.loc[nanInd, cat_dummies.columns.str.startswith('airportfrom_')]= np.nan
# cat_dummies

C:\Users\eilee\AppData\Local\Temp\ipykernel_1160\1814354145.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  cat_dummies.loc[nanInd, cat_dummies.columns.str.startswith('airportfrom_')]= np.nan


In [128]:
df = df.drop(['airline', 'airportfrom', 'airportto'], axis=1)
df = pd.concat([df, cat_dummies], axis=1)
# df

In [129]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
# df

c:\Users\eilee\anaconda3\envs\visual_inf\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\eilee\anaconda3\envs\visual_inf\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\eilee\anaconda3\envs\visual_inf\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\eilee\anaconda3\envs\visual_inf\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future

In [130]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
# df

c:\Users\eilee\anaconda3\envs\visual_inf\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\eilee\anaconda3\envs\visual_inf\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\eilee\anaconda3\envs\visual_inf\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\eilee\anaconda3\envs\visual_inf\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future

In [141]:
imputed = df.drop(['time', 'length', 'dayofweek', 'delay'], axis=1)
imputed = imputed[imputed.columns[imputed.columns.str.startswith('airportfrom_')]]
imputed = imputed.loc[nanInd]
imputed['airport_from'] = imputed.idxmax(axis=1)
# imputed['airportfrom']='' # to create an empty column
# for col_name in imputed.columns:
#     imputed.loc[imputed[col_name]==1,'airportfrom']= imputed['airportfrom']+' '+col_name
imputed = imputed['airport_from'].str.slice(start=12)
imputed

12      IAD
16      IDA
19      BNA
21      ATL
31      IAH
       ... 
4937    BHM
4966    DEN
4977    ORD
4992    DCA
4996    ELP
Name: airport_from, Length: 500, dtype: object

In [136]:
original = pd.read_csv("airlines.csv", nrows=5000)
original = original.loc[nanInd]['airportfrom']
original

12      LGB
16      LAX
19      SAV
21      FLL
31      EYW
       ... 
4937    BNA
4966    SLC
4977    ORD
4992    PIT
4996    BNA
Name: airportfrom, Length: 500, dtype: object

In [144]:
1-original.compare(imputed).shape[0]/original.shape[0]

0.22999999999999998